# OE62 Dataset Preprocessing

Generate LMDB files with structures and energy targets from the raw OE62 data [1]. As the raw DFT energies have a large absolute offset, perform a linear fit as outlined in Appendix A.10 of the Ewald message passing reference paper. Parts of this code are based on the original OE62 data analysis notebook.

In [10]:
import matplotlib.pyplot as plt
import lmdb
import pickle
from tqdm import tqdm
import torch
import os
from io import StringIO
from ase.io import read

from copy import deepcopy
import numpy as np
import pandas as pd

pd.set_option('display.width',5000)
pd.set_option('display.max_columns',200)

# Coulomb constants for electrostatic energy computation
ke = 14.399645351950548
kehalf = ke / 2

In [11]:
# Replace this by the directory containing the downloaded raw OE62 data
oe62_path = "oe62"
# Use the 62k dataframe containing the full OE62 dataset
df_path = os.path.join(oe62_path, "df_62k.json")
os.makedirs("oe62", exist_ok=True)

In [12]:
# Let's first load the dataframe into memory, this might take a moment.
# orient='split' keeps the column order as specified in table 2 of the publication.

df_62k = pd.read_json(df_path, orient='split') 

# Shape of the tabular dataframe returned in a tuple: (<number_of_rows>, <number_of_columns>)
print(df_62k.shape)

(61489, 29)


In [13]:
df_62k.columns

Index(['refcode_csd', 'canonical_smiles', 'inchi', 'number_of_atoms', 'xyz_pbe_relaxed', 'energies_occ_pbe', 'energies_occ_pbe0_vac_tier2', 'energies_occ_pbe0_water', 'energies_occ_pbe0_vac_tzvp', 'energies_occ_pbe0_vac_qzvp', 'energies_occ_gw_tzvp', 'energies_occ_gw_qzvp', 'cbs_occ_gw', 'energies_unocc_pbe', 'energies_unocc_pbe0_vac_tier2', 'energies_unocc_pbe0_water', 'energies_unocc_pbe0_vac_tzvp', 'energies_unocc_pbe0_vac_qzvp', 'energies_unocc_gw_tzvp', 'energies_unocc_gw_qzvp', 'cbs_unocc_gw', 'total_energy_pbe', 'total_energy_pbe0_vac_tier2', 'total_energy_pbe0_water', 'total_energy_pbe0_vac_tzvp', 'total_energy_pbe0_vac_qzvp', 'hirshfeld_pbe', 'hirshfeld_pbe0_vac_tier2', 'hirshfeld_pbe0_water'], dtype='object')

In [14]:
df_62k = df_62k.reset_index(drop=True)

To reproduce the binning experiments from the Ewald MP reference paper, rerun this notebook with d3fit == True (used to obtain the energy MAEs of the "cheating" baseline variation which does not have to learn D3 contributions)

In [34]:
d3fit = False

### Determine which atom types exist in OE62 and map each of them to a unique integer

In [35]:
from collections import Counter

all_symbols = []

for index, entry in tqdm(df_62k.iterrows(), total=len(df_62k)):
    atom_string = StringIO(entry["xyz_pbe_relaxed"])
    xyz_data = read(atom_string, format='xyz')
    symbols = xyz_data.get_chemical_symbols()
    for s in symbols:
        if s not in all_symbols:
            all_symbols.append(s)

100%|██████████| 61489/61489 [00:11<00:00, 5393.05it/s]


In [36]:
print(all_symbols)
num_elements = len(all_symbols)
print(num_elements)
index_map = dict(zip(all_symbols, range(num_elements)))
print(index_map)

['O', 'S', 'C', 'H', 'Cl', 'N', 'F', 'Br', 'Si', 'Se', 'P', 'B', 'I', 'Te', 'As', 'Li']
16
{'O': 0, 'S': 1, 'C': 2, 'H': 3, 'Cl': 4, 'N': 5, 'F': 6, 'Br': 7, 'Si': 8, 'Se': 9, 'P': 10, 'B': 11, 'I': 12, 'Te': 13, 'As': 14, 'Li': 15}


### Compute energies and regress targets

In [37]:
from sklearn.linear_model import LinearRegression
if d3fit:
    from dftd3.interface import DispersionModel, RationalDampingParam

regression_samples = []
targets = []
for index, entry in tqdm(df_62k.iterrows(), total=len(df_62k)):
    atom_string = StringIO(entry["xyz_pbe_relaxed"])

    xyz_data = read(atom_string, format='xyz')
    atomic_numbers = torch.Tensor(xyz_data.get_atomic_numbers())
    pos_bohr = torch.Tensor(xyz_data.get_positions())*1.8897259886
    
    # Compute DFT-D3 dispersion energy
    if d3fit:
        dispersion = DispersionModel(numbers=atomic_numbers.numpy(), 
                                 positions=pos_bohr.numpy())
        # Use recommended PBE0 BJ damping coefficients
        damp_param = RationalDampingParam(s6=1.0, a1=0.4145, s8=1.2177, a2=4.8593)
        e_vdw = dispersion.get_dispersion(damp_param, grad=False).get("energy")*27.211386245988
    else:
        e_vdw = 0

    symbols = xyz_data.get_chemical_symbols()
    counts = Counter(symbols)
    regression_sample = [0]*16
    for key, val in counts.items():
        regression_sample[index_map[key]] = val
    regression_samples.append(regression_sample)
    targets.append(-entry.total_energy_pbe0_vac_tier2 + e_vdw)
    

regression_samples = np.array(regression_samples, dtype=float)
targets = np.array(targets, dtype=float)

reg = LinearRegression(fit_intercept=True, n_jobs=4, positive=True).fit(regression_samples, targets)     

100%|██████████| 61489/61489 [01:03<00:00, 966.75it/s] 


In [38]:
ys_relaxed = -targets + reg.predict(regression_samples)

### Define train / validation / test split

In [39]:
df_62k_shuffled = df_62k.sample(frac=1, random_state=42)
n_train = 50000
n_val = 6000

### Target mean and standard deviation for corresponding "dataset" entry in experiment config files

In [40]:
print(ys_relaxed[df_62k_shuffled[:n_train].index].mean())

0.004212569638518444


In [41]:
print(ys_relaxed[df_62k_shuffled[:n_train].index].std())

1.7448031028122368


### Generate LMBDs

In [42]:
ds_name = "energy_linref_pbe0_d3fit" if d3fit == True else "energy_linref_pbe0"
os.makedirs(os.path.join("oe62", ds_name), exist_ok=True)

In [43]:
os.makedirs(os.path.join("oe62", ds_name, "train"), exist_ok=True)
db = lmdb.open(
    os.path.join("oe62", ds_name, "train", "pbe0_train.lmdb"),
    map_size=1099511627776 * 2,
    subdir=False,
    meminit=False,
    map_async=True,
)

In [44]:
from torch_geometric.data import Data

i=0
for index, entry in tqdm(df_62k_shuffled[:n_train].iterrows(), total=len(df_62k_shuffled[:n_train])):
    atom_string = StringIO(entry["xyz_pbe_relaxed"])
    xyz_data = read(atom_string, format='xyz')
    atomic_numbers = torch.Tensor(xyz_data.get_atomic_numbers())
    pos = torch.Tensor(xyz_data.get_positions())
    natoms = pos.shape[0]
    fixed = torch.zeros(natoms, dtype=torch.float32)
    sid = index
    refcode_csd = entry.refcode_csd
    y_relaxed = ys_relaxed[index]
    
    data = Data(
            pos=pos,
            atomic_numbers=atomic_numbers,
            natoms=natoms,
            sid=sid,
            refcode_csd=refcode_csd,
            fixed=fixed,
            y_relaxed = y_relaxed
        )
    
    txn = db.begin(write=True)
    txn.put(f"{i}".encode("ascii"), pickle.dumps(data, protocol=-1))
    txn.commit()
    db.sync()
    i+=1

db.close()     

100%|██████████| 50000/50000 [03:25<00:00, 243.52it/s]


In [45]:
os.makedirs(os.path.join("oe62", ds_name, "val"), exist_ok=True)
db = lmdb.open(
    os.path.join("oe62", ds_name, "val", "pbe0_val.lmdb"),
    map_size=1099511627776 * 2,
    subdir=False,
    meminit=False,
    map_async=True,
)

In [46]:
from torch_geometric.data import Data

i=0
for index, entry in tqdm(df_62k_shuffled[n_train:n_train+n_val].iterrows(), total=len(df_62k_shuffled[n_train:n_train+n_val])):
    atom_string = StringIO(entry["xyz_pbe_relaxed"])
    xyz_data = read(atom_string, format='xyz')
    atomic_numbers = torch.Tensor(xyz_data.get_atomic_numbers())
    pos = torch.Tensor(xyz_data.get_positions())
    natoms = pos.shape[0]
    fixed = torch.zeros(natoms, dtype=torch.float32)
    sid = index
    refcode_csd = entry.refcode_csd
    y_relaxed = ys_relaxed[index]
    
    data = Data(
            pos=pos,
            atomic_numbers=atomic_numbers,
            natoms=natoms,
            sid=sid,
            refcode_csd=refcode_csd,
            fixed=fixed,
            y_relaxed = y_relaxed,
        )
    
    txn = db.begin(write=True)
    txn.put(f"{i}".encode("ascii"), pickle.dumps(data, protocol=-1))
    txn.commit()
    db.sync()
    i+=1
    
db.close()     

100%|██████████| 6000/6000 [00:24<00:00, 246.14it/s]


In [47]:
os.makedirs(os.path.join("oe62", ds_name, "test"), exist_ok=True)
db = lmdb.open(
    os.path.join("oe62", ds_name, "test", "pbe0_test.lmdb"),
    map_size=1099511627776 * 2,
    subdir=False,
    meminit=False,
    map_async=True,
)

In [48]:
i = 0
for index, entry in tqdm(df_62k_shuffled[n_train+n_val:].iterrows(), total=len(df_62k_shuffled[n_train+n_val:])):
    atom_string = StringIO(entry["xyz_pbe_relaxed"])
    xyz_data = read(atom_string, format='xyz')
    atomic_numbers = torch.Tensor(xyz_data.get_atomic_numbers())
    pos = torch.Tensor(xyz_data.get_positions())
    natoms = pos.shape[0]
    fixed = torch.zeros(natoms, dtype=torch.float32)
    sid = index
    refcode_csd = entry.refcode_csd
    y_relaxed = ys_relaxed[index]
    
    data = Data(
            pos=pos,
            atomic_numbers=atomic_numbers,
            natoms=natoms,
            sid=sid,
            refcode_csd=refcode_csd,
            fixed=fixed,
            y_relaxed = y_relaxed
        )
    
    txn = db.begin(write=True)
    txn.put(f"{i}".encode("ascii"), pickle.dumps(data, protocol=-1))
    txn.commit()
    db.sync()
    i+=1

db.close()

100%|██████████| 5489/5489 [00:22<00:00, 244.54it/s]


### Store parameters of the energy fit in a json file

In [49]:
reg_info = {"Atom to Coefficient Mapping": dict(zip(all_symbols, range(num_elements))),
            "Regression Coefficients": list(reg.coef_),
            "Regression Intercept": float(reg.intercept_)}

import json

with open(os.path.join("oe62", ds_name, "offset_fitting_params_pbe0.json"), "w") as f:
    json.dump(reg_info, f)

with open(os.path.join("oe62", ds_name, "offset_fitting_params_pbe0.json")) as f:
    my_dict = json.load(f)

# Consistency check: this cell should output 0
reg2 = LinearRegression()
reg2.coef_=np.array(my_dict["Regression Coefficients"])
reg2.intercept_=np.array(my_dict["Regression Intercept"])

pred = ys_relaxed - reg2.predict(regression_samples)
print(np.mean(targets + pred))

0.0


# References

[1] Stuke, A., Kunkel, C., Golze, D., Todorovic, M., Margraf, ´
J. T., Reuter, K., Rinke, P., and Oberhofer, H. Atomic
structures and orbital energies of 61,489 crystal-forming
organic molecules. Scientific Data, 7(1):58, 2020.